In [1]:
# Import the dependencies.Numpy includes built-in random module and supplements.
import pandas as pd
import numpy as np

# Import the requests library.
import requests

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=200)
lngs = np.random.uniform(low=-180.000, high=180.000, size=200)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.(unzip)
coordinates = list(lat_lngs)

# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [3]:
# Create an empty list to hold the weather data.
city_data = []

# Create counters.
record_count = 1
set_count = 1

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Add 1 to the record count.
    i += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = (city_weather["coord"]["lat"])
        city_lng = (city_weather["coord"]["lon"])
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country":city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
    except KeyError:
        continue
len(city_data)

126

In [4]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Richards Bay,ZA,-28.7830,32.0377,74.48,82,0,12.93,clear sky
1,Rikitea,PF,-23.1203,-134.9692,80.47,73,39,8.34,scattered clouds
2,Flinders,AU,-34.5833,150.8552,64.99,89,100,7.00,light rain
3,Dahuk,IQ,36.8695,42.9940,45.10,70,1,3.24,clear sky
4,Nyimba,ZM,-14.5566,30.8149,68.74,90,82,4.88,broken clouds
5,Tokmak,UA,47.2514,35.7058,32.90,88,72,5.59,broken clouds
6,Dnestrovsc,MD,46.6222,29.9133,41.00,100,90,6.71,overcast clouds
7,Jamestown,US,42.0970,-79.2353,35.60,32,1,8.48,clear sky
8,Khatanga,RU,71.9667,102.5000,-20.58,94,66,12.71,broken clouds
9,Bengkulu,ID,-3.8004,102.2655,75.74,79,99,3.87,overcast clouds


In [5]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")